In Python, STAPLE is available through SimpleITK.

Reference: https://towardsdatascience.com/how-to-use-the-staple-algorithm-to-combine-multiple-image-segmentations-ce91ebeb451e


## Packages

In [ ]:
!pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 30.4 MB/s eta 0:00:00


In [ ]:
import SimpleITK as sitk # https://simpleitk.org/
from matplotlib import pyplot as plt
import os
import numpy as np
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Params

In [ ]:
PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/landmark_segmentation/eval/'
IMG_PATH = PROJECT_PATH+ 'base_images/'
MSK_PATH = PROJECT_PATH + 'masks/'
STAPLE_PATH = PROJECT_PATH + 'staple/'
STAPLE_BIN_PATH = PROJECT_PATH + 'staple_bin/'

In [ ]:
type_filters = ['artificial', 'transport', 'urban', 'water', 'nature', 'all']
zone_filters = ['zone01', 'zone02', 'zone03', 'zone04', 'zone05', 'zone06', 'zone07', 'zone08', 'zone09', 'zone10',
                'zone11', 'zone12', 'zone13', 'zone14', 'zone15', 'zone16', 'zone17', 'zone18', 'zone19', 'zone20']

## Preprocessing

In [ ]:
# My masks are in LA format (with transparency), so I need to convert them to a Numpy array with PIL
def itk_readmask(mask_path):
  # Load the image using PIL and convert it to a NumPy array
  pil_image = Image.open(mask_path).convert('L')  # 'L' mode for 8-bit grayscale
  image_array = np.array(pil_image)
  # Create a SimpleITK image from the NumPy array
  return sitk.GetImageFromArray(image_array)

In [ ]:
# Annotation inputs for the STAPLE algorithm need to be binary
def binarize_sitk(image):
    return sitk.BinaryThreshold(image, lowerThreshold=1, upperThreshold=255, insideValue=1, outsideValue=0)

## STAPLE

In [ ]:
# This function allow to filter the image and mask names
def filter_names(im_names, keyword):
  return [name for name in im_names if keyword in name]

In [ ]:

def apply_staple(annotation_paths):
    annotations = []
    for annotation_path in annotation_paths:
        annot = itk_readmask(annotation_path)
        annot = binarize_sitk(annot)
        annotations.append(annot)
    # Apply STAPLE algorithm
    staple_filter = sitk.STAPLEImageFilter()
    staple_filter.SetForegroundValue(1) # input segmentations consist of 1's everywhere inside the segmented region
    consensus_segmentation = staple_filter.Execute(annotations)
    return consensus_segmentation

In [ ]:
def compute_staple(annot_folder, output_folder, type_filters, zone_filters):
  annot_names = os.listdir(annot_folder)
  # For each zone and each anchor type
  for z in zone_filters:
      for t in type_filters:
          # filter annotations
          annot_filtered = filter_names(annot_names, z)
          annot_filtered = filter_names(annot_filtered, t)
          paths = [annot_folder + name for name in annot_filtered]
          # apply staple
          consensus_segmentation = apply_staple(paths)
          # convert from float between 0 and 1 to UInt8 between 0 and 255
          consensus_segmentation = consensus_segmentation * 255
          consensus_segmentation = sitk.Cast(consensus_segmentation, sitk.sitkUInt8)
          # save consensus as png in output_folder
          sitk.WriteImage(consensus_segmentation, output_folder + z + '_' + t + '.png')
          print(f"Done {z}_{t}")


In [ ]:
compute_staple(MSK_PATH, STAPLE_PATH, type_filters, zone_filters)

Done zone01_artificial
Done zone01_transport
Done zone01_urban
Done zone01_water
Done zone01_nature
Done zone01_all
Done zone02_artificial
Done zone02_transport
Done zone02_urban
Done zone02_water
Done zone02_nature
Done zone02_all
Done zone03_artificial
Done zone03_transport
Done zone03_urban
Done zone03_water
Done zone03_nature
Done zone03_all
Done zone04_artificial
Done zone04_transport
Done zone04_urban
Done zone04_water
Done zone04_nature
Done zone04_all
Done zone05_artificial
Done zone05_transport
Done zone05_urban
Done zone05_water
Done zone05_nature
Done zone05_all
Done zone06_artificial
Done zone06_transport
Done zone06_urban
Done zone06_water
Done zone06_nature
Done zone06_all
Done zone07_artificial
Done zone07_transport
Done zone07_urban
Done zone07_water
Done zone07_nature
Done zone07_all
Done zone08_artificial
Done zone08_transport
Done zone08_urban
Done zone08_water
Done zone08_nature
Done zone08_all
Done zone09_artificial
Done zone09_transport
Done zone09_urban
Done zone

## Evaluation

In [ ]:
def visualize_sitk_image(sitk_image, title="Image"):
    # Convert ITK image to a NumPy array for visualization
    image_array = sitk.GetArrayViewFromImage(sitk_image)
    # Visualize the image using matplotlib
    plt.imshow(image_array, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
def plot_histogram(image):
    image_array = sitk.GetArrayFromImage(image)*255
    plt.hist(image_array.ravel(), bins=range(int(np.max(image_array)) + 2), align='left')
    plt.title('Histogram of Segmentation Values')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.show()

## Threshold to get binary staple

In [ ]:
# for each image in staple folder, use a threshold to convert values to 0 if under 250 and 255 else

for file in os.listdir(STAPLE_PATH):
    # open image with PIL
    pil_image = Image.open(STAPLE_PATH + file).convert('L')  # 'L' mode for 8-bit grayscale
    # convert all pixel values inferior to 100 to 0 and above to 255
    pil_image = pil_image.point(lambda x: 0 if x < 127 else 255)
    # save in staple_bin folder
    pil_image.save(STAPLE_BIN_PATH + file)

## Eval staple binary

In [ ]:
image_path = "/content/drive/MyDrive/Colab Notebooks/landmark_segmentation/eval/staple_bin/zone01_all.png"  # Replace with the actual path
image = Image.open(image_path).convert('L')  # Open in grayscale mode
image_np = np.array(image)  # Convert to NumPy array

In [ ]:
unique_values = np.unique(image_np)
print(unique_values)

[  0 255]


In [ ]:
# Count pixels of value 0
count_0 = np.count_nonzero(image_np == 0)
# Count pixels of value 255
count_255 = np.count_nonzero(image_np == 255)

print(f"Number of pixels with value 0: {count_0}")
print(f"Number of pixels with value 255: {count_255}")

Number of pixels with value 0: 1387003
Number of pixels with value 255: 337157


In [ ]:
# Count number of pixels drawn by staple for each eval zone
pixel_counts = []

for file in os.listdir(STAPLE_PATH):
    # Charger l'image avec numpy
    image = np.array(Image.open(STAPLE_BIN_PATH + file))
    # Calculer le nombre de pixels annotés
    pixels = np.count_nonzero(image)
    pixel_counts.append((file, pixels))

In [ ]:
total_pixels_per_zone = 1920*898

In [ ]:
# Afficher les résultats
for file, pixels in pixel_counts:
    print(f"Image: {file}, Nombre de pixels annotés: {pixels}")
    print(f"Ratio: {pixels/total_pixels_per_zone}")
    print()

Image: zone01_artificial.png, Nombre de pixels annotés: 11327
Ratio: 0.006569575909428359

Image: zone01_transport.png, Nombre de pixels annotés: 180982
Ratio: 0.10496821640682999

Image: zone01_urban.png, Nombre de pixels annotés: 120509
Ratio: 0.06989432535263548

Image: zone01_water.png, Nombre de pixels annotés: 92040
Ratio: 0.053382516703786194

Image: zone01_nature.png, Nombre de pixels annotés: 0
Ratio: 0.0

Image: zone01_all.png, Nombre de pixels annotés: 337157
Ratio: 0.19554855697847068

Image: zone02_artificial.png, Nombre de pixels annotés: 70247
Ratio: 0.04074273849294729

Image: zone02_transport.png, Nombre de pixels annotés: 179287
Ratio: 0.10398512899034892

Image: zone02_urban.png, Nombre de pixels annotés: 199693
Ratio: 0.11582045749814403

Image: zone02_water.png, Nombre de pixels annotés: 117984
Ratio: 0.06842984409799555

Image: zone02_nature.png, Nombre de pixels annotés: 64750
Ratio: 0.03755451930215293

Image: zone02_all.png, Nombre de pixels annotés: 388861
Rat